In [63]:
import pytesseract
from tqdm.auto import tqdm
import os
import numpy as np
import chromadb
from sentence_transformers import SentenceTransformer
import pinecone

In [67]:
pinecone.init(api_key = "b60565d6-28e0-41c4-9be4-a77fd78d892c",environment = "us-west1-gcp-free")

In [45]:
dir_name = 'docs-sm'

In [99]:
files_path=[]
for dir in os.listdir("docs-sm/"):
    
    for idx,subdir in enumerate(os.listdir(os.path.join(dir_name,dir))):
        files_path.append(os.path.join(os.path.join(dir_name,dir),subdir))
        idx+=1
        if idx==100:
            break

10 docs extracted

In [100]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [101]:
ocr_texts=[]
for image in tqdm(files_path):
    text = pytesseract.image_to_string(image)
    ocr_texts.append(text)

100%|██████████| 1600/1600 [15:02<00:00,  1.77it/s]


In [83]:
import pandas as pd
df = pd.DataFrame({'OCR Text': ocr_texts})
df.head()

,OCR Text
0,
1,stone coviow: sie\n\nUANN-ERICKSON RESP EP\n\n...
2,E\n_ VANTAGE VANTAGE\n\nSEossotes |\n
3,57% of Winston smokers\nfind clean tasting Fac...
4,"Ls latest US. Government figures\n\n-""PALLMATL..."


In [84]:
df['File Path'] =files_path

In [85]:
df['File Category']= df['File Path'].apply(lambda x:x.split("\\")[1])

In [86]:
df['OCR Text'] = df['OCR Text'].replace("",np.nan)


In [87]:
df.dropna(inplace=True)

In [88]:
df.drop(['File Path'],inplace=True,axis=1)

In [89]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [90]:
db_text=list(df['OCR Text'].values)
category = list(df['File Category'].values)
df

,OCR Text,File Category
1,stone coviow: sie\n\nUANN-ERICKSON RESP EP\n\n...,advertisement
2,E\n_ VANTAGE VANTAGE\n\nSEossotes |\n,advertisement
3,57% of Winston smokers\nfind clean tasting Fac...,advertisement
4,"Ls latest US. Government figures\n\n-""PALLMATL...",advertisement
5,Fee}\nDd\n=\n=\niS\nka\nrs\nzZ\nEd\n\n,advertisement
...,...,...
315,|THE Ae som\nAMERICAN PRODUCTION SPECIFICATION...,specification
316,PRODUCTION SPE\n\nIFICATIONS\n\n10F2\nCONFIDEN...,specification
317,SPECIFICATION CHANGE PROPOSAL NO. 90 - 184\nBR...,specification
318,csssent) aoe & WILL HNsoN Toon comPoRATION ee ...,specification


In [94]:
embedding  = model.encode("This is a sentence")
print(embedding.tolist())
# pinecone.create_index('ocr-text',dimension=384)

[0.05048312619328499, 0.08800595253705978, 0.0048748827539384365, 0.03626894950866699, -0.0010181255638599396, 0.019153712317347527, 0.014231867156922817, -0.018786577507853508, 0.08152502030134201, 0.03721952065825462, 0.07041557133197784, -0.058707378804683685, -0.016879601404070854, -0.006291268393397331, -0.006761717610061169, 0.016697537153959274, 0.048855971544981, -0.019675064831972122, -0.0874321386218071, -0.004071816336363554, -0.02663280814886093, 0.0924726352095604, -0.004529280122369528, 0.00046385181485675275, -0.015113696455955505, 0.009122522547841072, -0.0432257205247879, 0.06419794261455536, 0.06789980828762054, -0.020825596526265144, -0.05017518624663353, -0.011976719833910465, 0.07046797126531601, 0.047732237726449966, 0.02371334098279476, -0.013724074698984623, -0.005713066551834345, 0.0422920323908329, 0.013804353773593903, -0.012826891615986824, 0.01073297206312418, -0.038728512823581696, 0.017663301900029182, 0.0045972419902682304, 0.009400886483490467, -0.00647

In [95]:
index = pinecone.Index('ocr-text')
id = index.describe_index_stats()['total_vector_count']
id

151

In [57]:
# client = chromadb.Client()
# collection = client.create_collection("ocr-project")

In [96]:
questions_list = []
for i, row in tqdm(df.iterrows()):
    vector_id = str(id + i)
    questions_list.append(
        (
            vector_id,
            model.encode(row['OCR Text']).tolist(),
            {
                
                'OCR_Text': row['OCR Text'],
                'category': row['File Category']
            }
        )
    )

    index.upsert(vectors = questions_list)


300it [03:07,  1.60it/s]


In [58]:
# for i, row in tqdm(df.iterrows()):
#     collection.add(
#         documents=[row['OCR Text']],
#         metadatas=[{'category':row['File Category']}],
#         ids=[str(i)]

#     )

151it [00:11, 13.56it/s]


In [59]:
# with open('ocr_result.txt', 'r') as file:
#     ocr_text = file.read()
results = collection.query(
    query_texts=[ocr_text],
    n_results=1
)

In [60]:
results

{'ids': [['127']],
 'embeddings': None,
 'documents': [['CURRICULUM VITAE\n\nNAME: RUDOLF EDMOND NOBLE\nDATE OF BIRTH: July 8, 1933\nEDUCATION:\n\nB.S., University of California, Berkeley ‘california,\n. (organic Chemistry) 1953.\n\nPh.D. ,University of Colorado, Boulder, Colorado :\n» ° (Organic Chemistry) 1956 ‘s “\nE.I. du Pont de Nemours & Co., Ince, |v p\n; Pre-Doctoral Fellow. «= port a sti\n\nM.D., Western Reserve University, Cleveland, Onto, 1963.\n\nStraight Medical Internship, University of California ieee\nHospital, San Francisco, 1963-1964.\n\nguntor Assistant Medical Residency, Stanford Hospitals,\nPalo Alto, California, 1964 -1965.\n\nRESEARCH EXPERIENCE: i : ‘\n1956-57: Fulbright post-Doctoral Research Petiow, 2\n. University of Heidelberg, Heidelberg, Germany,\n\n1957-59: Research Chemist, California Research Conp+, 3\nRichmond, California. : ad\n\n1959-63: Research Fellow in Meéicine, Western Reserve\nUniversity, Cleveland, Ohlo.\n\n1965-67: Research Fellow in Medicine

In [61]:
category = results['metadatas'][0][0]['category']
print(category)

resume


In [ ]:
import json
with open('category.txt', 'w') as file:
    json.dump(category, file)

In [62]:
pip install pinecone-client


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

                                              0.0/179.1 kB ? eta -:--:--
     ------                                   30.7/179.1 kB ? eta -:--:--
     --------------------                    92.2/179.1 kB 1.0 MB/s eta 0:00:01
     ------------------------------         143.4/179.1 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 179.1/179.1 kB 1.1 MB/s eta 0:00:00
                                              0.0/60.0 kB ? eta -:--:--
     ---------------------------------------- 60.0/60.0 kB 1.6 MB/s eta 0:00:00
                                              0.0/283.7 kB ? eta -:--:--
     --------                                61.4/283.7 kB 1.7 MB/s eta 0:00:01
     ----------------                       122.9/283.7 kB 1.8 MB/s eta 0:00:01
     ------------------------------         225.3/283.7 kB 1.7 MB/s eta 0:00:01
    